In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import gc
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [ ]:
train_dir = '../input/state-farm-distracted-driver-detection/imgs/train'
test_dir = '../input/state-farm-distracted-driver-detection/imgs/test'
X = []
y = []
for dirname, _, filenames in os.walk(train_dir):
    for filename in filenames:
        X.append(cv2.resize(cv2.imread(os.path.join(dirname, filename),0),(512,512),interpolation = cv2.INTER_CUBIC))
        y.append(os.path.join(dirname,filename)[60])    
        

In [ ]:
c = list(zip(X, y))
random.shuffle(c)
X, y =zip(*c)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape,y.shape

In [ ]:
X = X.reshape(22424,512,512,1)
y = y.reshape(22424,1)

In [ ]:
from keras.utils import to_categorical
y = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.10,random_state=2)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_train, y_train, test_size=0.10,random_state=2)

In [ ]:
del X
del y
gc.collect()

X_train.shape,X_test.shape, X_val.shape

In [ ]:
ntrain = len(X_train)
nval = len(X_val)

batch_size = 64

In [ ]:
from keras import layers,models,optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array,load_img

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation = 'relu',input_shape=(512,512,1)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.001),metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2,
                                  height_shift_range=0.2,zoom_range=0.2,shear_range=0.2,
                                  horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow(X_train,y_train,batch_size=batch_size)
val_generator = val_datagen.flow(X_val,y_val,batch_size=batch_size)

In [ ]:
history = model.fit_generator(train_generator,step_per_epoch= ntrain//batch_size,epochs=20,validation_data=val_generator,
                             validation_steps= nval//batch_size)

In [ ]:
X_test = X_test/255.0
test_loss, test_acc = model.evaluate(X_test,y_test)

In [ ]:
print(test_loss,test_acc)